In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import time
import json
import pickle
import warnings

import numpy as np
import pandas as pd
from nilearn.image import load_img
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix
from nilearn.reporting import make_glm_report

sys.path.append('/home/ubuntu/repos/learning-habits-analysis')
from utils.data import Subject, load_participant_list, create_dummy_regressors, collapse_events

In [23]:
base_dir = '/home/ubuntu/data/learning-habits'
bids_dir = "/home/ubuntu/data/learning-habits/bids_dataset/derivatives/fmriprep-24.0.1"

sub_ids = load_participant_list(base_dir)

In [24]:
#subjects = [Subject(base_dir, sub_id, include_modeling=True, include_imaging=True, bids_dir=bids_dir) for sub_id in sub_ids]
subject = Subject(base_dir, '02', include_modeling=True, include_imaging=True, bids_dir=bids_dir)

In [25]:
run = 'test'

In [26]:
model_params = {
    'model_name': 'stim_category_collapsed',
    'tr': 2.33384,
    'hrf_model': 'spm',
    'noise_model': 'ar1',
    'smoothing_fwhm': 5,
    'motion_type': 'basic',
    'include_physio': True,
    'brain_mask': True,
    'fd_thresh': 0.5,
    'std_dvars_thresh': 2,
    'exclusion_threshold': 0.2,
    'scrub': 'dummies',
    'duration': 'all',
    'collapse_events': True,
    'iti_included': False
}

In [ ]:
# Parameters
model_name = model_params['model_name']
tr = model_params['tr']
hrf_model = model_params['hrf_model']
noise_model = model_params['noise_model']
smoothing_fwhm = model_params['smoothing_fwhm']
motion_type = model_params['motion_type']
include_physio = model_params['include_physio']
fd_thresh = model_params['fd_thresh']
std_dvars_thresh = model_params['std_dvars_thresh']
exclusion_threshold = model_params['exclusion_threshold']
scrub = model_params['scrub']
brain_mask = model_params['brain_mask']
duration = model_params['duration']
collapse_events = model_params['collapse_events']
iti_included = model_params['iti_included']

In [8]:
# Load confounds
confounds, sample_mask = subject.load_confounds(run, motion_type=motion_type,
                                                fd_thresh=fd_thresh, std_dvars_thresh=std_dvars_thresh,
                                                scrub=(0 if scrub == 'dummies' else scrub))
if include_physio:
    physio_regressors = subject.load_physio_regressors(run)
    confounds = confounds.join(physio_regressors)

In [9]:
if scrub == 'dummies':
    dummies = create_dummy_regressors(sample_mask, len(confounds))
    confounds = pd.concat([confounds, dummies], axis=1)

In [10]:
# Load fMRI volume
img_path = subject.img.get(run)
fmri_img = load_img(img_path)

In [11]:
if brain_mask:
    brain_mask_path = subject.brain_mask.get(run)
    brain_mask = load_img(brain_mask_path)
else:
    brain_mask = None

n = fmri_img.shape[-1]
frametimes = np.linspace(tr / 2., (n - .5) * tr, n)

In [19]:
events = getattr(subject, run).extend_events_df(columns_event={'first_stim_cat':'first_stim_presentation'})

In [13]:
# Add stim category to first stim presentation
events['trial_type'] = events.apply(
    lambda row: f"{row['trial_type']}_{row['first_stim_cat']}" if row['trial_type'] == 'first_stim_presentation' else row['trial_type'],
    axis=1
)

In [20]:
events.head(15)

,onset,duration,trial_type,trial,first_stim_cat
0,0.009226,0.817357,first_stim_presentation,1,figure
1,0.826583,0.364869,second_stim_presentation,1,None
2,1.191453,0.000000,response,1,None
3,1.210246,2.151995,purple_frame,1,None
4,3.362241,7.240122,iti,1,None
5,10.602363,0.934128,first_stim_presentation,2,house
6,11.536491,0.329929,second_stim_presentation,2,None
7,11.866420,0.000000,response,2,None
8,11.870114,2.185362,purple_frame,2,None
9,14.055476,6.222508,iti,2,None


In [21]:
collapse_events(events).head()

,onset,duration,trial_type,trial,first_stim_cat
0,0.009226,1.182227,stim_presentation,1,figure
1,1.191453,2.151995,feedback,1,figure
2,3.362241,7.240122,iti,1,None
3,10.602363,1.264057,stim_presentation,2,house
4,11.866420,2.185362,feedback,2,house


In [ ]:
# Ignore warnings related to null duration events and unexpected columns in events data
warnings.filterwarnings("ignore", message=".*events with null duration.*")
warnings.filterwarnings("ignore", message=".*following unexpected columns in events data.*")
# Create design matrix
design_matrix = make_first_level_design_matrix(frame_times=frametimes,
                                    events=events,
                                    hrf_model=hrf_model,
                                    drift_model=None,
                                    high_pass=high_pass,
                                    add_regs=confounds)

In [ ]:
# Select the first 8 columns and the first 100 rows
subset_design_matrix = design_matrix.iloc[:100, :8]

# Plot the subset of the design matrix
subset_design_matrix.plot(figsize=(12, 8))
plt.legend(loc='best')
plt.title('First 8 Columns of the Design Matrix (First 100 Rows)')
plt.xlabel('Time Points')
plt.ylabel('Values')
plt.show()

In [ ]:
# Compute the average correlation matrices
average_correlation_matrix = design_matrix.iloc[:, :15].corr().values

# Create subplots
fig, axes = plt.subplots(1, 1, figsize=(10, 8))

# Plot the correlation matrix for design_matrix
sns.heatmap(average_correlation_matrix, xticklabels=design_matrix.columns[:15], 
            yticklabels=design_matrix.columns[:15], cmap='coolwarm', annot=True, ax=axes)
axes.set_title('Average Correlation Matrix (design_matrix)')

plt.show()